In [1]:

using Gridap
using LinearAlgebra
# using Test
using Gridap.ODEs, Gridap.Algebra


In [86]:

domain = (-1, +1, -1, +1)
partition = (50, 50)
model = CartesianDiscreteModel(domain, partition)


CartesianDiscreteModel()

In [87]:
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)

V_1 = TestFESpace(model, reffe,
conformity=:H1)
V_2 = TestFESpace(model, reffe,
conformity=:H1)

UnconstrainedFESpace()

In [88]:
U_1 = TransientTrialFESpace(V_1)
U_2 = TransientTrialFESpace(V_2)

TrialFESpace()

In [89]:

X = TransientMultiFieldFESpace([U_1, U_2])
Y = MultiFieldFESpace([V_1, V_2])

MultiFieldFESpace()

In [90]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
Γ = Boundary(model,tags="boundary")
dΓ = Measure(Γ,degree)

GenericMeasure()

In [157]:

λ_0(t) = x ->  (2 + cos(x[1]/0.5 * 2 * pi)) * 0.1 
λ(t, u_other) =  λ_0(t) * (u_other+0.1)
f(t) = x -> (0.5 + sin(t*2) ) * exp(-(x[1]^2 + x[2]^2) * 100) 
h(x) = 0.0

h (generic function with 1 method)

In [164]:
m(t, dtu, v) = ∫(v * dtu)dΩ
a(t, u, v) = ∫(λ_0(t) * ∇(v) ⋅ ∇(u))dΩ
b(t, v) =  ∫(v * f(t))dΩ + ∫( v*h )dΓ 
res(t, (u1, u2), (v1, v2)) = m(t,∂t(u1),v1) + m(t,∂t(u2),v2) + a(t, u1, v1) + a(t, u2, v2) - b(t, v1) - b(t, v2)

jac(t,(u1,u2),(du1,du2),(v1,v2)) = a(t, du1, v1) + a(t, du2, v2)
jac_t(t,(u1,u2),(du1t,du2t),(v1,v2)) = m(t,du1t,v1) + m(t,du2t,v2)
# l(t, v) = 
op = TransientFEOperator(res, (jac, jac_t), X, Y)

TransientFEOpFromWeakForm()

In [165]:
lin_solver = LUSolver()
nl_solver = NLSolver(lin_solver, method=:newton, iterations=50, show_trace=false)
# nls = NewtonRaphsonSolver(lin_solver,1.0e-6,10)
Δt = 0.02
θ = 0.5
ode_solver = ThetaMethod(nl_solver, Δt, θ)
# solver_1 = TransientFESolver(ode_solver)
# For a two-stage singly-diagonally-implicit scheme (of order 2), it would be

# tableau = :SDIRK_2_2
# solver_rk = RungeKutta(nl_solver, lin_solver, Δt, tableau)

ThetaMethod()

In [166]:
# ls = LUSolver()
# Δt = 0.05
# θ = 0.5
# solver = ThetaMethod(ls, Δt, θ)
# tableau = :SDIRK_2_2
# solver_rk = RungeKutta(ls, ls, Δt, tableau)

In [167]:
u1_0(t) = x -> 0.0
u2_0(t) = x -> 0.0

u2_0 (generic function with 1 method)

In [168]:
# t0, tF = 0.0, 10.0
U10 = U_1(0.0)
U20 = U_2(0.0)
X0 = X(0.0)
u10t(t) = x -> 0.0 * t
u10 = TimeSpaceFunction(u10t)
u20t(t) = x -> 0.0 * t
u20 = TimeSpaceFunction(u20t)

u1h0 = interpolate_everywhere(u10(0.0),U10)
u2h0 = interpolate_everywhere(u20(0.0),U20)
xh0 = interpolate_everywhere([u1h0,u2h0],X0)
sol_t = solve(ode_solver, op, 0.0, 10.0, xh0)
# for (sl, t) in uh
    
# end

GenericTransientFESolution()

In [169]:
if !isdir("tmp2")
    mkdir("tmp2")
  end
  
  createpvd("results2") do pvd
    pvd[0] = createvtk(Ω, "tmp2/results_0" * ".vtu", cellfields=["u1" => u1h0, "u2" => u2h0])
    # t = 1
    cnt = 1
    for (t, sl) in sol_t
      # print(item[1])
      if cnt % 10 == 0
      pvd[t] = createvtk(Ω, "tmp2/results_$t" * ".vtu", cellfields=["u1" => sl[1], "u2" => sl[2]])
      end
      cnt +=1
    end
  end

52-element Vector{String}:
 "results2.pvd"
 "tmp2/results_0.vtu"
 "tmp2/results_0.19999999999999998.vtu"
 "tmp2/results_0.4000000000000001.vtu"
 "tmp2/results_0.6000000000000002.vtu"
 "tmp2/results_0.8000000000000004.vtu"
 "tmp2/results_1.0000000000000004.vtu"
 "tmp2/results_1.2000000000000006.vtu"
 "tmp2/results_1.4000000000000008.vtu"
 "tmp2/results_1.600000000000001.vtu"
 "tmp2/results_1.8000000000000012.vtu"
 "tmp2/results_2.0000000000000013.vtu"
 "tmp2/results_2.2000000000000015.vtu"
 ⋮
 "tmp2/results_7.799999999999922.vtu"
 "tmp2/results_7.999999999999917.vtu"
 "tmp2/results_8.199999999999914.vtu"
 "tmp2/results_8.39999999999991.vtu"
 "tmp2/results_8.599999999999905.vtu"
 "tmp2/results_8.799999999999901.vtu"
 "tmp2/results_8.999999999999897.vtu"
 "tmp2/results_9.199999999999893.vtu"
 "tmp2/results_9.399999999999888.vtu"
 "tmp2/results_9.599999999999884.vtu"
 "tmp2/results_9.79999999999988.vtu"
 "tmp2/results_9.999999999999876.vtu"

In [76]:
uh.odesltn.us0[1]

242-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [69]:
t

TrialFESpace()

In [44]:
for (t, state) in Gridap.ODEs.each_time_step(uh)

end

LoadError: UndefVarError: `each_time_step` not defined in `Gridap.ODEs`
Suggestion: check for spelling errors or missing imports.

In [45]:
for sol in uh
    # Assuming `sol` is a tuple of (time, state)
    t, state = sol
end

LoadError: This function belongs to an interface definition and cannot be used.

In [37]:
# x = get_cell_points(Ω)

In [46]:
methods(uh)

# 0 methods for callable object

In [56]:
unh1, unh2 = uh.trial(0)

MultiFieldFESpace()

In [62]:
# unh1.space.fe_basis

SingleFieldFEBasis():
 num_cells: 100
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 18393536350112245733

In [39]:
for (sl, t) in uh
    
end

LoadError: Function get_data is not implemented for MultiFieldCellField at this moment.
You need to extract the individual fields and then evaluate them separately.

If ever implement this, evaluating a `MultiFieldCellField` directly would provide,
at each evaluation point, a tuple with the value of the different fields.


LoadError: This function belongs to an interface definition and cannot be used.

In [95]:

# using Gridap
# using LinearAlgebra
# using Test
# using Gridap.ODEs, Gridap.Algebra
# # using GridapDistributed
# # using PartitionedArrays

# # function main(distribute,parts)
#   # ranks = distribute(LinearIndices((prod(parts),)))

# θ = 0.5

# ut(t) = x -> VectorValue(x[1],x[2])*t
# u = TimeSpaceFunction(ut)

# pt(t) = x -> (x[1]-x[2])*t
# p = TimeSpaceFunction(pt)
# q(x) = t -> p(t,x)

# ft(t) = x -> ∂t(u)(t,x) - Δ(u)(t,x) + ∇(p)(t,x)
# gt(t) = x -> (∇⋅u)(t,x)
# ht(t) = x -> ∇(u)(t,x)⋅VectorValue(0.0,1.0) - p(t,x)*VectorValue(0.0,1.0)
# f = TimeSpaceFunction(ft)
# g = TimeSpaceFunction(gt)
# h = TimeSpaceFunction(ht)

# domain = (0,1,0,1)
# partition = (4,4)
# model = CartesianDiscreteModel(domain,partition)
# labels = get_face_labeling(model)
# add_tag_from_tags!(labels,"neumann",6)
# add_tag_from_tags!(labels,"dirichlet",[1,2,3,4,5,7,8])

# order = 2

# reffeᵤ = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
# V0 = FESpace(
#   model,
#   reffeᵤ,
#   conformity=:H1,
#   dirichlet_tags="dirichlet"
# )

# reffeₚ = ReferenceFE(lagrangian,Float64,order-1)
# Q = TestFESpace(
#   model,
#   reffeₚ,
#   conformity=:H1
# )

# U = TransientTrialFESpace(V0,u)

# P = TrialFESpace(Q)

# Ω = Triangulation(model)
# degree = 2*order
# dΩ = Measure(Ω,degree)

# Γ = Boundary(model,tags=["neumann"])
# dΓ = Measure(Γ,degree)

# #
# a(t,u,v) = ∫(∇(u)⊙∇(v))dΩ
# b(t,(v,q)) = ∫(v⋅f(t))dΩ + ∫(q*g(t))dΩ + ∫(v⋅h(t))dΓ
# m(t,ut,v) = ∫(ut⋅v)dΩ

# X = TransientMultiFieldFESpace([U,P])
# Y = MultiFieldFESpace([V0,Q])

# res(t,(u,p),(v,q)) = a(t,u,v) + m(t,∂t(u),v) - ∫((∇⋅v)*p)dΩ + ∫(q*(∇⋅u))dΩ - b(t,(v,q))
# jac(t,(u,p),(du,dp),(v,q)) = a(t,du,v) - ∫((∇⋅v)*dp)dΩ + ∫(q*(∇⋅du))dΩ
# jac_t(t,(u,p),(dut,dpt),(v,q)) = m(t,dut,v)

# U0 = U(0.0)
# P0 = P(0.0)
# X0 = X(0.0)
# uh0 = interpolate_everywhere(u(0.0),U0)
# ph0 = interpolate_everywhere(p(0.0),P0)
# xh0 = interpolate_everywhere([uh0,ph0],X0)

# op = TransientFEOperator(res,(jac,jac_t),X,Y)

# t0 = 0.0
# tF = 1.0
# dt = 0.1

# ls  = LUSolver()
# nls = NewtonRaphsonSolver(ls,1.0e-6,10)
# ode_solver = ThetaMethod(nls,dt,θ)

# sol_t = solve(ode_solver,op,t0,tF,xh0)

# l2(w) = w⋅w
# tol = 1.0e-6
# for (tn, xh_tn) in sol_t
#   uh_tn = xh_tn[1]
#   ph_tn = xh_tn[2]
#   #writevtk(Ω,"output/tmp_stokes_OB_sol_$tn.vtu",cellfields=["u"=>uh_tn,"p"=>ph_tn])
#   e = u(tn) - uh_tn
#   el2 = sqrt(sum(∫(l2(e))dΩ))
#   e = p(tn) - ph_tn
#   el2 = sqrt(sum(∫(l2(e))dΩ))
#   @test el2 < tol
# end
# # end

# # end #module

LoadError: invalid redefinition of constant Main.f